In [1]:
import numpy as np

In [2]:
# generate random patters of ltl

# 1. unary operators G, F, R, W, X, ~
# 2. binary operators  &, |, →, ↔     (include true and false?)
# 3. variables a, b, c, d....

# generate formulas as random nodes in a tree
# for each node certain prob of of node being populated

## Generating random LTL formulas as binary trees

In [3]:
binary_operators = ['&', '|', '→', '↔']
binary_dist = np.array([0.35, 0.35, 0.2, 0.10])

unary_operators = ['G', 'F', 'R', '~']
unary_dist = np.array([0.2, 0.2, 0.2, 0.4])

variables = ['a', 'b', 'c', 'd']
variable_dist = np.array([0.4, 0.35, 0.15, 0.10]) # logarithmic probabilities

symbols = binary_operators + unary_operators + variables
symbol_dist = np.concatenate([binary_dist * len(binary_dist)/len(symbols), #P(symbol) = P(class) * P(symbol | class)
                              unary_dist * len(unary_dist)/len(symbols),
                              variable_dist * len(variable_dist)/len(symbols)]) # weight for None symbol

symbol_dist /= np.sum(symbol_dist) # making the dist sum up to 1

class LTLfomula:
    def __init__(self, symbol, depth, max_depth):
        self.symbol = symbol
        self.depth = depth
        self.max_depth = max_depth

        self.left = None
        self.right = None

        # left child
        if self.symbol not in unary_operators: # unary symbols can have only one child
            self.left = self.sample_child()
        # right child
        self.right = self.sample_child()


    def sample_child(self):
        if self.depth == self.max_depth or self.symbol in variables:
            return None
        elif self.depth == self.max_depth - 1: # the next layer is the last one -> generate variable
            return LTLfomula(sample_variable(), self.depth + 1, self.max_depth)
        else:
            return LTLfomula(sample_symbol(), self.depth + 1, self.max_depth)

def sample_variable():
    return np.random.choice(variables, size=1, p=variable_dist)[0]

def sample_symbol():
    return np.random.choice(symbols, size=1, p=symbol_dist)[0]

def generate_random_LTL(max_depth):
    while True:
        root_symbol = sample_symbol()
        if root_symbol in variables: # filter out only propositions
            continue
        tree = LTLfomula(root_symbol, depth=1, max_depth=max_depth)
        break

    return tree

## Parsing tree representation into formula

In [4]:
def insert_at(s, insert, cursor):
    return s[:cursor] + insert + s[cursor:]

def print_formula_tree(tree):
    def recursive_print_node(node, output="", cursor=0): # modified in order traversal
        if node is not None and node.symbol is not None:
            output, cursor = recursive_print_node(node.left, output, cursor)

            output = insert_at(output, node.symbol, cursor)
            cursor += 1

            if node.symbol in unary_operators:
                output = insert_at(output, "()", cursor)
                cursor += 1

            output, cursor = recursive_print_node(node.right, output, cursor)

            #if node.symbol in binary_operators:
            #    output = insert_at(output, "(", cursor)
             #   output = insert_at(output, ")", cursor)
            #    cursor += 1

        return output, cursor

    output, _ = recursive_print_node(tree)
    return output

In [5]:
for i in range(50):
    tree = generate_random_LTL(3)
    print(print_formula_tree(tree))

a→b&~(a)
c&b→a
G(G(d))
b|a&a
F(G(b))
b&~(d)
d|b↔R(a)
b↔~(c)
c|a→b
G(b&a|a)
R(c|G(b))
~(b|G(a))
G(b)
b|a|a|a
b↔a
R(a|a)
F(c↔b&b)
G(d→a)
~(R(c))
~(b&~(b))
F(R(b))
~(c|b)
~(F(a))
a&a&b
G(b)
~(d|a)
~(R(b))
~(a&b)
F(G(a))
~(a&a)
~(a|b)
~(a|F(a))
F(a|b)
c|b|b
~(~(a))
F(b&a)
~(F(b))
a|G(b)
a&a
c|a↔a|a
G(c↔b)
R(~(a))
c↔a&a&b
R(~(a))
a&c&c
G(c)
R(a)
b&~(b)
G(F(c))
~(a)


Translating formula into "pseudo-natural" language

In [6]:
binary_operators = ['&', '|', '→', '↔']
binary_dist = np.array([0.35, 0.35, 0.2, 0.10])

unary_operators = ['G', 'F', 'R', '~']
unary_dist = np.array([0.2, 0.2, 0.2, 0.4])

variables = ['a', 'b', 'c', 'd', 'e', 'f']
variable_dist = np.array([0.3, 0.21, 0.15, 0.13, 0.11, 0.10]) # logarithmic probabilities

symbol2words = {
    '&': "and",
    '|': "or",
    '→': "implies",
    '↔': "iff and only if",
    'G': "it always holds that",
    'F': "finally it holds that", #?????
    'R': "release", #???
    '~': "it's not true that"
}

In [7]:
for i in range(10):
    tree = generate_random_LTL(3)
    formula = print_formula_tree(tree)
    output = ""
    for symbol in formula:
        if symbol in symbol2words.keys():
            output += symbol2words[symbol] + " "

        if symbol in variables:
            output += symbol + " holds "

    print(output)

b holds implies f holds or b holds 
finally it holds that a holds 
it always holds that it always holds that a holds 
release b holds 
it always holds that e holds or b holds 
it's not true that b holds and a holds iff and only if e holds 
c holds and a holds iff and only if d holds or f holds 
a holds or a holds or b holds 
release c holds 
it's not true that f holds or b holds 
